# Multilayer Perceptron

Trong bài này, sinh viên được hướng dẫn sử dụng Tensorflow để hiện thực Multilayer Perceptron (neural network).

Code được tham khảo từ một số nguồn sau đây:
* [Aymeric Damien (aymericdamien)](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py)

Dataset: the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/). Links: [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).

In [ ]:
from __future__ import print_function

import tensorflow as tf

In [10]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

# load dataset
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [ ]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

**Cách đầu tiên, sử dụng các phép toán đơn giản để định nghĩa MLP**

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

** Định nghĩa MLP gọn hơn với các phép toán đã được tensorflow hỗ trợ sẵn, ở đây ta dùng *tf.layers.dense* **

*Sinh viên chỉ dùng một trong 2 version của hàm multilayer_perceptron, ở đây hoặc ở phía trên*

In [ ]:
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, units=n_hidden_1, kernel_initializer=tf.contrib.layers.xavier_initializer())
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, units=n_hidden_2, kernel_initializer=tf.contrib.layers.xavier_initializer())
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, units=n_classes, kernel_initializer=tf.contrib.layers.xavier_initializer())
    return out_layer

Ở đây, thay vì khởi tạo các matrix weights và biases với random_normal, ta dùng xavier_initializer, nó có ảnh hưởng nhất định tới tốc độ hội tụ của model

Ở đây, chúng ta bắt đầu cấu trúc graph, định nghĩa loss (cost) và optimizer để đưa vào training

In [ ]:
# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

Train model và test để xem kết quả

In [16]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 cost=0.380549035
Epoch: 0002 cost=0.318104575
Epoch: 0003 cost=0.304515623
Epoch: 0004 cost=0.296059326
Epoch: 0005 cost=0.289836827
Epoch: 0006 cost=0.285455442
Epoch: 0007 cost=0.282269923
Epoch: 0008 cost=0.278459263
Epoch: 0009 cost=0.275646231
Epoch: 0010 cost=0.273887377
Epoch: 0011 cost=0.271059293
Epoch: 0012 cost=0.271695899
Epoch: 0013 cost=0.268166179
Epoch: 0014 cost=0.265963621
Epoch: 0015 cost=0.266773281
Epoch: 0016 cost=0.263803870
Epoch: 0017 cost=0.261205883
Epoch: 0018 cost=0.259613837
Epoch: 0019 cost=0.259056695
Epoch: 0020 cost=0.258419550
Epoch: 0021 cost=0.258958084
Epoch: 0022 cost=0.258818751
Epoch: 0023 cost=0.255654335
Epoch: 0024 cost=0.255509429
Epoch: 0025 cost=0.253457383
Epoch: 0026 cost=0.253793295
Epoch: 0027 cost=0.252277589
Epoch: 0028 cost=0.251613997
Epoch: 0029 cost=0.251272571
Epoch: 0030 cost=0.250379945
Optimization Finished!
Accuracy: 0.9216


## Bài tập

Bài nộp của sinh viên là chính là **file này** sau khi được đổi tên thành **MSSV.E10_Multilayer_Perceptron.ipynb** và đừng quên ghi thông tin sinh viên vào các ô ở dưới.

Địa chỉ nộp bài: https://www.dropbox.com/request/cmGOid3eqfC87NDza002

Deadline nộp bài: **10:00 thứ 2 tuần tiếp theo**

*Điểm bài này sẽ được tổng hợp với điểm chấm trên lớp (nếu có) để ra điểm cuối cùng, do đó sinh viên vắng học trên lớp dù làm bài có toàn vẹn cũng có thể nhận được điểm thấp hơn điểm tối đa*

Thông tin sinh viên:

In [6]:
sid = '2013452'
name = 'Đào Quốc Khánh'

import tensorflow as tf
import numpy as np

# sinh viên import các thư viện cần thiết ở đây
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10

tf.compat.v1.disable_eager_execution()

### Bài 1
Trong ví dụ ở trên, chúng ta đã sử dụng toàn bộ tập dữ liệu để train, và đồng thời cũng lấy chính dữ liệu đó để test, điều đó dẫn tới trường hợp "học khớp" (overfitting). 

Sinh viên hãy chia tập dữ liệu ra làm 2 phần riêng biệt ~90% để train và ~10% còn lại dùng để test. (sửa trực tiếp ở code ví dụ ở trên)

Hãy cho biết kích thước của mỗi tập sau khi tách ra

Hãy cho biết các nhận xét của kết quả khi chia ra 2 tập rời như vậy

Do có sự cải tiến về phiên bản của thư viện, việc hiện thực code như ví dụ trên có thể không còn phù hợp với một số phiên bản thư viện hiện tại. Do đó ở đây chúng ta sẽ hiện thực một bản code mới phù hợp hơn:

In [7]:
# Load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

"""
print(x_train.shape[0])
print(x_test.shape[0])
"""

# Split into TRAIN and VALID
train_len = 0.9 * x_train.shape[0] # using 90% dataset for training
valid_len = x_train.shape[0] - int(train_len)

x_val = x_train[int(train_len):x_train.shape[0]]
x_train = x_train[0:int(train_len)]
y_val = y_train[int(train_len):y_train.shape[0]]
y_train = y_train[0:int(train_len)]

print("Length of training set:", x_train.shape[0])
print("Length of training set:", x_val.shape[0])

"""
print("train data has " + str(x_train.shape[0]) + " sample")
print("every train data is " + str(x_train.shape[1]) + ' * ' + str(x_train.shape[2]) + " image")

print("validation data has " + str(x_val.shape[0]) + " sample")
print("every validation data is " + str(x_val.shape[1]) + ' * ' + str(x_val.shape[2]) + " image")
"""

# Reshape
x_train = x_train.reshape(int(train_len), 784)
x_val = x_val.reshape(valid_len, 784)

"""
print(x_train.shape)
print(x_val.shape)
"""

# Normalize data (Optional)
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale

# Label to one hot encoding value
num_classes = 10

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)

# Parameter
learning_rate = 0.01
epoch = 30
batch_size = 1000
iteration = len(x_train) / batch_size

# MLP Graph
x = tf.compat.v1.placeholder(tf.float32, [None, 784])
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

def mlp(x):
    # hidden layer 1:
    w1 = tf.Variable(tf.compat.v1.random_uniform([784, 256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer 2:
    w2 = tf.Variable(tf.compat.v1.random_uniform([256, 256]))
    b2 = tf.Variable(tf.zeros([256]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # output layer
    w3 = tf.Variable(tf.compat.v1.random_uniform([256, 10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits = tf.matmul(h2, w3) + b3
    
    return logits

# Construct model, define loss and optimizer
logits = mlp(x)

loss_op = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y))

train_op = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_op)

# Initialize the variables
init = tf.compat.v1.global_variables_initializer()

# Start training
with tf.compat.v1.Session() as sess:
    # Run the init
    sess.run(init)
    
    for ep in range(epoch):
        avg_loss = 0
        start, end = 0, batch_size
        
        for i in range(int(iteration)):
            _, loss = sess.run([train_op, loss_op],
                              feed_dict = {x: x_train[start:end], y: y_train[start:end]})
            start += batch_size
            end += batch_size
            # Compute average loss
            avg_loss += loss / iteration
        
        print("Epoch: " + str(ep) + " cost = " + str(avg_loss))
    
    print("Optimization Finished!")
    
    # Test model
    preds = tf.nn.softmax(logits) # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.compat.v1.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: ", accuracy.eval({x: x_val, y: y_val}))

Length of training set: 54000
Length of training set: 6000
Epoch: 0 cost = 14524.989266854747
Epoch: 1 cost = 542.7926160671094
Epoch: 2 cost = 21.109659654122815
Epoch: 3 cost = 12.533292134602863
Epoch: 4 cost = 9.750126308865017
Epoch: 5 cost = 7.880049422935206
Epoch: 6 cost = 6.604579510512174
Epoch: 7 cost = 5.739798236776282
Epoch: 8 cost = 5.0766453522223
Epoch: 9 cost = 4.752056395566021
Epoch: 10 cost = 4.599619719717238
Epoch: 11 cost = 3.8927306290026062
Epoch: 12 cost = 3.4065068121309636
Epoch: 13 cost = 3.485653331986181
Epoch: 14 cost = 3.7890743988531606
Epoch: 15 cost = 3.1424919499291315
Epoch: 16 cost = 3.1933937668800354
Epoch: 17 cost = 2.9524654635676635
Epoch: 18 cost = 2.7529445996990916
Epoch: 19 cost = 2.232393966780769
Epoch: 20 cost = 2.0073621692480867
Epoch: 21 cost = 4.0262272291713295
Epoch: 22 cost = 2.4878352328583047
Epoch: 23 cost = 2.2299668810985707
Epoch: 24 cost = 1.8122375132860964
Epoch: 25 cost = 1.5611681353162843
Epoch: 26 cost = 1.39291070

Sau khi tách, kích thước tập dữ liệu dùng để train là 54000 và kích thước tập dữ liệu dùng để test là 6000.

Hai tập dữ liệu này được lấy từ cùng một tập dữ liệu ban đầu, vậy nên chúng gần như có cùng phân phối cũng như các tính chất, thuộc tính căn bản.

Sau quá trình huấn luyện, sử dụng tập dữ liệu test để kiểm tra kết quả, ta thu được độ chính xác của mô hình: Accuracy = 0.936.

### Bài 2
Hãy sử dụng tập dữ liệu [cifar 10](https://www.cs.toronto.edu/~kriz/cifar.html)

Hãy xử lý dữ liệu, thay đổi model (nếu cần) và điều chỉnh các thông số cần thiết phù hợp với dữ liệu này.

Sinh viên hãy train, test và cho biết kết quả đạt được (yêu cầu giữ kết quả output khi chạy train, test lúc nộp bài ở các cell tiếp theo ở dưới

Sau đây chúng ta sẽ sử dụng 2 mô hình: Multilayer Perceptron (MLP) và Convolutional Neural Network (CNN) để huấn luyện tập dữ liệu cũng như so sánh hiệu quả giữa chúng.

### MLP 

In [18]:
# data load, pre-processing
# Load data
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding = 'bytes')
    return dict

file = []
file.append('D:\Khanh\Code\Tổng Hợp AI\IASLab\cifar-10-batches-py\data_batch_1')
file.append('D:\Khanh\Code\Tổng Hợp AI\IASLab\cifar-10-batches-py\data_batch_2')
file.append('D:\Khanh\Code\Tổng Hợp AI\IASLab\cifar-10-batches-py\data_batch_3')
file.append('D:\Khanh\Code\Tổng Hợp AI\IASLab\cifar-10-batches-py\data_batch_4')
file.append('D:\Khanh\Code\Tổng Hợp AI\IASLab\cifar-10-batches-py\data_batch_5')
file_test = r'D:\Khanh\Code\Tổng Hợp AI\IASLab\cifar-10-batches-py\test_batch'

data_batch = []
for i in range(5):
    data_batch.append(unpickle(file[i]))
    
test_batch = unpickle(file_test)

# Preprocessing
x_train = []
y_train = []
for i in range(5):
    x_train.append(data_batch[i][b'data']) 
    y_train.append(data_batch[i][b'labels'])
    
x_test = test_batch[b'data']
y_test = test_batch[b'labels']

# Normalize data
scale = 255
for i in range(5):
    x_train[i] = x_train[i].astype('float32')
    x_train[i] /= scale
    
x_test = x_test.astype('float32')
x_test /= scale

# Label to one hot encoding value
num_classes = 10

for i in range(5):
    y_train[i] = tf.keras.utils.to_categorical(y_train[i], num_classes)
    
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Parameter
learning_rate = 0.005
epoch = 1000
n_hidden_layer_1 = 256
n_hidden_layer_2 = 128

In [19]:
# model (if need)
x = tf.compat.v1.placeholder(tf.float32, [None, 3072])
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

def mlp(x):
    # hidden layer 1:
    w1 = tf.Variable(tf.compat.v1.random_uniform([3072, n_hidden_layer_1]))
    b1 = tf.Variable(tf.zeros([n_hidden_layer_1]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer 2:
    w2 = tf.Variable(tf.compat.v1.random_uniform([n_hidden_layer_1, n_hidden_layer_2]))
    b2 = tf.Variable(tf.zeros([n_hidden_layer_2]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # output layer
    w3 = tf.Variable(tf.compat.v1.random_uniform([n_hidden_layer_2, 10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits = tf.matmul(h2, w3) + b3
    
    return logits

# Construct model, define loss and optimizer
logits = mlp(x)

loss_op = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y))

train_op = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_op)

# Initialize the variables
init = tf.compat.v1.global_variables_initializer()

In [20]:
# train, test (keep output when submit your exercise)
with tf.compat.v1.Session() as sess:
    # Run the init
    sess.run(init)
    
    for ep in range(epoch):
        avg_loss = 0
        
        for i in range(5):
            _, loss = sess.run([train_op, loss_op],
                              feed_dict = {x: x_train[i], y: y_train[i]})
            # Compute average loss
            avg_loss += loss / 5
        
        print("Epoch: " + str(ep) + " cost = " + str(avg_loss))
    
    print("Optimization Finished!")
    
    # Test model
    preds = tf.nn.softmax(logits) # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.compat.v1.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test}))

Epoch: 0 cost = 219638.10000000003
Epoch: 1 cost = 173809.259375
Epoch: 2 cost = 161408.3171875
Epoch: 3 cost = 112164.8828125
Epoch: 4 cost = 76703.1640625
Epoch: 5 cost = 60393.317968749994
Epoch: 6 cost = 70911.23203125
Epoch: 7 cost = 53196.6203125
Epoch: 8 cost = 38586.54609375
Epoch: 9 cost = 32278.170703124997
Epoch: 10 cost = 21465.778515625003
Epoch: 11 cost = 26478.8046875
Epoch: 12 cost = 29232.866015624997
Epoch: 13 cost = 25066.828515625
Epoch: 14 cost = 28488.15078125
Epoch: 15 cost = 28596.763281249998
Epoch: 16 cost = 26243.264453125
Epoch: 17 cost = 24578.53046875
Epoch: 18 cost = 24927.179296875
Epoch: 19 cost = 19473.1564453125
Epoch: 20 cost = 21483.978906250002
Epoch: 21 cost = 23526.544140625
Epoch: 22 cost = 19336.953125
Epoch: 23 cost = 16556.4076171875
Epoch: 24 cost = 16220.0130859375
Epoch: 25 cost = 20137.656640625
Epoch: 26 cost = 21361.887499999997
Epoch: 27 cost = 17627.836523437498
Epoch: 28 cost = 17426.9490234375
Epoch: 29 cost = 11258.60048828125
Epoc

Epoch: 232 cost = 199.9800262451172
Epoch: 233 cost = 174.86481018066408
Epoch: 234 cost = 165.12868041992186
Epoch: 235 cost = 140.28195037841797
Epoch: 236 cost = 146.75325775146484
Epoch: 237 cost = 160.82906188964844
Epoch: 238 cost = 135.53815917968748
Epoch: 239 cost = 152.37051696777343
Epoch: 240 cost = 151.0531692504883
Epoch: 241 cost = 146.40413208007814
Epoch: 242 cost = 131.46524658203126
Epoch: 243 cost = 128.0245101928711
Epoch: 244 cost = 153.5078872680664
Epoch: 245 cost = 129.89347381591796
Epoch: 246 cost = 166.14923095703125
Epoch: 247 cost = 230.58943786621091
Epoch: 248 cost = 270.79982910156247
Epoch: 249 cost = 212.2179473876953
Epoch: 250 cost = 345.8343566894531
Epoch: 251 cost = 285.73809814453125
Epoch: 252 cost = 216.1260986328125
Epoch: 253 cost = 175.3041534423828
Epoch: 254 cost = 155.85847930908204
Epoch: 255 cost = 160.79115600585936
Epoch: 256 cost = 225.02392578125
Epoch: 257 cost = 169.73317260742186
Epoch: 258 cost = 190.7660369873047
Epoch: 259 co

Epoch: 459 cost = 35.17971649169921
Epoch: 460 cost = 37.006177520751955
Epoch: 461 cost = 43.51173324584961
Epoch: 462 cost = 55.59696273803711
Epoch: 463 cost = 56.43189392089844
Epoch: 464 cost = 36.16020393371582
Epoch: 465 cost = 30.524234008789062
Epoch: 466 cost = 31.08536643981934
Epoch: 467 cost = 24.504800415039064
Epoch: 468 cost = 25.08454284667969
Epoch: 469 cost = 23.451205062866208
Epoch: 470 cost = 20.94368953704834
Epoch: 471 cost = 20.442467498779298
Epoch: 472 cost = 22.902515411376953
Epoch: 473 cost = 27.537580871582026
Epoch: 474 cost = 34.40705337524414
Epoch: 475 cost = 29.955075836181642
Epoch: 476 cost = 27.407397842407228
Epoch: 477 cost = 22.660707092285158
Epoch: 478 cost = 21.376859283447267
Epoch: 479 cost = 19.088336181640624
Epoch: 480 cost = 17.7531946182251
Epoch: 481 cost = 15.852370643615725
Epoch: 482 cost = 15.36570224761963
Epoch: 483 cost = 15.896186256408692
Epoch: 484 cost = 18.60349292755127
Epoch: 485 cost = 25.71071357727051
Epoch: 486 cost

Epoch: 684 cost = 11.414434814453125
Epoch: 685 cost = 13.809432220458984
Epoch: 686 cost = 17.98043746948242
Epoch: 687 cost = 20.15825881958008
Epoch: 688 cost = 23.620944786071778
Epoch: 689 cost = 28.53842620849609
Epoch: 690 cost = 29.16730537414551
Epoch: 691 cost = 30.395117950439452
Epoch: 692 cost = 21.553376388549804
Epoch: 693 cost = 19.823379325866696
Epoch: 694 cost = 13.11750602722168
Epoch: 695 cost = 14.390313339233398
Epoch: 696 cost = 12.222812271118164
Epoch: 697 cost = 10.800682067871096
Epoch: 698 cost = 10.624524879455567
Epoch: 699 cost = 8.628169822692872
Epoch: 700 cost = 8.180843162536622
Epoch: 701 cost = 7.365031909942626
Epoch: 702 cost = 6.965396785736084
Epoch: 703 cost = 6.702623176574707
Epoch: 704 cost = 6.601852226257323
Epoch: 705 cost = 6.461101913452149
Epoch: 706 cost = 6.371498489379883
Epoch: 707 cost = 6.279651069641114
Epoch: 708 cost = 6.2328346252441404
Epoch: 709 cost = 6.219810581207275
Epoch: 710 cost = 6.131459426879882
Epoch: 711 cost =

Epoch: 911 cost = 5.066332721710205
Epoch: 912 cost = 4.988902282714844
Epoch: 913 cost = 5.634746074676514
Epoch: 914 cost = 5.38331184387207
Epoch: 915 cost = 6.928554344177246
Epoch: 916 cost = 5.874066066741943
Epoch: 917 cost = 5.080212974548339
Epoch: 918 cost = 5.205313491821289
Epoch: 919 cost = 7.5089668273925785
Epoch: 920 cost = 6.072758769989013
Epoch: 921 cost = 6.192804145812988
Epoch: 922 cost = 6.256783294677734
Epoch: 923 cost = 9.366892242431641
Epoch: 924 cost = 11.264114570617675
Epoch: 925 cost = 7.897206497192382
Epoch: 926 cost = 6.397406387329101
Epoch: 927 cost = 5.854257106781006
Epoch: 928 cost = 5.728135108947754
Epoch: 929 cost = 7.222020816802979
Epoch: 930 cost = 6.417772769927978
Epoch: 931 cost = 7.549678611755371
Epoch: 932 cost = 7.451142597198486
Epoch: 933 cost = 9.031379222869873
Epoch: 934 cost = 7.490642929077148
Epoch: 935 cost = 8.16272268295288
Epoch: 936 cost = 7.460245800018311
Epoch: 937 cost = 7.649176120758057
Epoch: 938 cost = 6.57961702

...

Epoch: 2441 cost = 1.067473602294922
Epoch: 2442 cost = 1.067099690437317
Epoch: 2443 cost = 1.0765750408172607
Epoch: 2444 cost = 1.0756659269332887
Epoch: 2445 cost = 1.0628870487213133
Epoch: 2446 cost = 1.0519973039627075
Epoch: 2447 cost = 1.0425740838050843
Epoch: 2448 cost = 1.0239542961120605
Epoch: 2449 cost = 1.0264369487762453
Epoch: 2450 cost = 1.0139384746551512
Epoch: 2451 cost = 1.0068210244178772
Epoch: 2452 cost = 1.0059062004089356
Epoch: 2453 cost = 1.0063077569007874
Epoch: 2454 cost = 1.0069405078887939
Epoch: 2455 cost = 1.0049776673316955
Epoch: 2456 cost = 0.9888825416564943
Epoch: 2457 cost = 0.9884210348129273
Epoch: 2458 cost = 0.983724570274353
Epoch: 2459 cost = 0.9799086451530457
Epoch: 2460 cost = 0.9802717328071594
Epoch: 2461 cost = 0.9911849856376648
Epoch: 2462 cost = 1.0126745462417601
Epoch: 2463 cost = 1.0410000801086425
Epoch: 2464 cost = 1.0533942222595214
Epoch: 2465 cost = 1.0269694089889525
Epoch: 2466 cost = 1.0119027972221375
Epoch: 2467 cos

Epoch: 2658 cost = 1.0151552557945251
Epoch: 2659 cost = 1.013719093799591
Epoch: 2660 cost = 1.0030967473983765
Epoch: 2661 cost = 0.9980017185211181
Epoch: 2662 cost = 1.0072700381278992
Epoch: 2663 cost = 0.9932138919830322
Epoch: 2664 cost = 0.9918559908866882
Epoch: 2665 cost = 0.9902312636375428
Epoch: 2666 cost = 1.000154435634613
Epoch: 2667 cost = 1.0126912355422975
Epoch: 2668 cost = 1.0276874065399169
Epoch: 2669 cost = 1.035952877998352
Epoch: 2670 cost = 1.0390787839889526
Epoch: 2671 cost = 1.0324556827545166
Epoch: 2672 cost = 1.0294129848480225
Epoch: 2673 cost = 1.0295776486396788
Epoch: 2674 cost = 1.0349288940429688
Epoch: 2675 cost = 1.0321513414382935
Epoch: 2676 cost = 1.0326954126358032
Epoch: 2677 cost = 1.0360496759414672
Epoch: 2678 cost = 1.0432966947555542
Epoch: 2679 cost = 1.0394623279571533
Epoch: 2680 cost = 1.0259649515151978
Epoch: 2681 cost = 1.0158521652221681
Epoch: 2682 cost = 1.0329618930816649
Epoch: 2683 cost = 1.03427939414978
Epoch: 2684 cost 

Epoch: 2876 cost = 1.0773444175720215
Epoch: 2877 cost = 1.0837172031402589
Epoch: 2878 cost = 1.1263949632644652
Epoch: 2879 cost = 1.1426539421081543
Epoch: 2880 cost = 1.1233745098114014
Epoch: 2881 cost = 1.1095572233200073
Epoch: 2882 cost = 1.094634771347046
Epoch: 2883 cost = 1.0887169361114502
Epoch: 2884 cost = 1.06650230884552
Epoch: 2885 cost = 1.0606489896774292
Epoch: 2886 cost = 1.0562510251998902
Epoch: 2887 cost = 1.053543782234192
Epoch: 2888 cost = 1.0483190774917603
Epoch: 2889 cost = 1.0387680053710937
Epoch: 2890 cost = 1.0430845499038697
Epoch: 2891 cost = 1.044131898880005
Epoch: 2892 cost = 1.0380868196487427
Epoch: 2893 cost = 1.0733173131942748
Epoch: 2894 cost = 1.1250418424606323
Epoch: 2895 cost = 1.1021064519882202
Epoch: 2896 cost = 1.1107664823532104
Epoch: 2897 cost = 1.110810351371765
Epoch: 2898 cost = 1.0963496685028076
Epoch: 2899 cost = 1.0765554428100585
Epoch: 2900 cost = 1.0669899225234984
Epoch: 2901 cost = 1.0595863580703737
Epoch: 2902 cost =

Epoch: 3094 cost = 1.0735980749130247
Epoch: 3095 cost = 1.0663492202758789
Epoch: 3096 cost = 1.0780262470245363
Epoch: 3097 cost = 1.1039178848266602
Epoch: 3098 cost = 1.1039339542388915
Epoch: 3099 cost = 1.0671045780181885
Epoch: 3100 cost = 1.0494443655014039
Epoch: 3101 cost = 1.0384944200515749
Epoch: 3102 cost = 1.0338961839675904
Epoch: 3103 cost = 1.0272278785705566
Epoch: 3104 cost = 1.0141038179397583
Epoch: 3105 cost = 1.0067404866218566
Epoch: 3106 cost = 1.0065893173217775
Epoch: 3107 cost = 1.0169600129127503
Epoch: 3108 cost = 1.0073720693588255
Epoch: 3109 cost = 1.0078403472900392
Epoch: 3110 cost = 1.009589719772339
Epoch: 3111 cost = 1.0176485419273378
Epoch: 3112 cost = 1.0166199326515197
Epoch: 3113 cost = 1.0194695472717286
Epoch: 3114 cost = 1.02792329788208
Epoch: 3115 cost = 1.0336904287338256
Epoch: 3116 cost = 1.0426990509033203
Epoch: 3117 cost = 1.0532557725906373
Epoch: 3118 cost = 1.033331072330475
Epoch: 3119 cost = 1.0453968048095703
Epoch: 3120 cost

Epoch: 3311 cost = 1.0242522597312926
Epoch: 3312 cost = 1.0343139171600342
Epoch: 3313 cost = 1.0245344638824463
Epoch: 3314 cost = 1.0271095514297484
Epoch: 3315 cost = 1.044848370552063
Epoch: 3316 cost = 1.0624633073806764
Epoch: 3317 cost = 1.1435913801193236
Epoch: 3318 cost = 1.1981821060180664
Epoch: 3319 cost = 1.2115241050720214
Epoch: 3320 cost = 1.2126307725906371
Epoch: 3321 cost = 1.18038170337677
Epoch: 3322 cost = 1.1587156295776366
Epoch: 3323 cost = 1.131710124015808
Epoch: 3324 cost = 1.118001389503479
Epoch: 3325 cost = 1.1123040914535522
Epoch: 3326 cost = 1.164332890510559
Epoch: 3327 cost = 1.3371859312057495
Epoch: 3328 cost = 1.2534985780715941
Epoch: 3329 cost = 1.2076140403747557
Epoch: 3330 cost = 1.17941792011261
Epoch: 3331 cost = 1.1468846321105957
Epoch: 3332 cost = 1.1390580654144287
Epoch: 3333 cost = 1.131306529045105
Epoch: 3334 cost = 1.115297245979309
Epoch: 3335 cost = 1.1083636522293092
Epoch: 3336 cost = 1.1122368574142456
Epoch: 3337 cost = 1.1

Ở trên, sinh viên sử dụng mô hình Multilayer Perceptron cùng với tập dữ liệu dùng để train lấy từ các file "data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5" và tập dữ liệu test lấy từ file "test_batch".

Do những hạn chế về thời gian và phần cứng sử dụng, hiện tại sinh viên chưa thể chọn được các tham số learning rate và epoch phù hợp để mô hình trở nên tốt nhất (nếu được sẽ cập nhật lại các tham số này trong thời gian sớm nhất), cho nên độ chính xác hiện tại của mô hình chỉ vào khoảng: Accuracy = 0.4389.

Tuy nhiên, tư tưởng và mã nguồn hiện thực mô hình đối với dữ liệu được yêu cầu đã được trình bày đầy đủ ở trên.

### CNN

In [7]:
# data load, pre-processing
# Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data
scale = 255
x_train = x_train.astype('float32') / scale
x_test = x_test.astype('float32') / scale

# Label to one hot encoding value
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
# model (if need)
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,
                padding = 'same', activation = 'relu', input_shape = x_train.shape[1:]))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 32, kernel_size = 2, 
                 padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(filters = 64, kernel_size = 2, 
                 padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation = 'softmax'))

# Compile the model
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop', metrics = ['accuracy'])

In [9]:
# train, test (keep output when submit your exercise)
# Start training
hist = model.fit(x_train, y_train, 
                 batch_size = 100, epochs = 30, 
                 verbose = 2, shuffle = True)

print("Optimization Finished!")

# Calculate accuracy
score = model.evaluate(x_test, y_test, verbose = 0)
print('Accuracy:', score[1])

Train on 50000 samples
Epoch 1/30
50000/50000 - 33s - loss: 1.6980 - accuracy: 0.3858 - 33s/epoch - 660us/sample
Epoch 2/30
50000/50000 - 33s - loss: 1.3783 - accuracy: 0.5066 - 33s/epoch - 662us/sample
Epoch 3/30
50000/50000 - 31s - loss: 1.2439 - accuracy: 0.5567 - 31s/epoch - 622us/sample
Epoch 4/30
50000/50000 - 30s - loss: 1.1476 - accuracy: 0.5925 - 30s/epoch - 607us/sample
Epoch 5/30
50000/50000 - 30s - loss: 1.0845 - accuracy: 0.6158 - 30s/epoch - 610us/sample
Epoch 6/30
50000/50000 - 31s - loss: 1.0247 - accuracy: 0.6377 - 31s/epoch - 620us/sample
Epoch 7/30
50000/50000 - 31s - loss: 0.9827 - accuracy: 0.6540 - 31s/epoch - 616us/sample
Epoch 8/30
50000/50000 - 32s - loss: 0.9448 - accuracy: 0.6692 - 32s/epoch - 638us/sample
Epoch 9/30
50000/50000 - 32s - loss: 0.9085 - accuracy: 0.6817 - 32s/epoch - 634us/sample
Epoch 10/30
50000/50000 - 30s - loss: 0.8828 - accuracy: 0.6923 - 30s/epoch - 605us/sample
Epoch 11/30
50000/50000 - 27s - loss: 0.8567 - accuracy: 0.7011 - 27s/epoch 

### Bài 3
Các bạn đã train để tạo ra được model sử dụng trong classification đơn giản, tuy nhiên chúng ta chưa lưu lại các kết quả đạt được để tái sử dụng, do đó, mỗi lần test lại sẽ cần train lại mô hình, tốn kém thời gian.

Hãy tìm hiểu và hiện thực (vào code mẫu ở trên) để lưu lại model và tái sử dụng lại sau này.

In [17]:
tf.test.gpu_device_name()

'/device:GPU:0'

Ta sẽ lưu các trọng số đạt được của mô hình vào file "cifar10_cnn_model.hdf5" để tái sử dụng trong những lần sau.

In [10]:
# Saving weights
model.save_weights('cifar10_model.hdf5')

# Loading weights and Testing
model.load_weights('cifar10_model.hdf5')

score = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy:', score[1])

Accuracy: 0.7422
